<a href="https://colab.research.google.com/github/sazio/Transients/blob/master/Python/DynamicTimeWarping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install dtaidistance

     |████████████████████████████████| 757kB 6.0MB/s 
  Created wheel for dtaidistance: filename=dtaidistance-2.1.2-cp36-cp36m-linux_x86_64.whl size=1446280 sha256=0e068891244f062c5288beb383794c1da225fc1576ef385f58ceb91716bfd259
  Stored in directory: /root/.cache/pip/wheels/d6/2c/9e/ed7ddb5538a7f3eab7b743e657f539260c46252c330ee0180e
Successfully built dtaidistance


In [15]:
from dtaidistance import dtw, dtw_ndim
from dtaidistance import dtw_visualisation as dtwvis
import numpy as np

In [22]:
# Fast C version
s1 = np.array([0, 0, 1, 2, 1, 0, 1, 0, 0], dtype=np.double)
s2 = np.array([0.0, 1, 2, 0, 0, 0, 0, 0, 0], dtype=np.double)
d = dtw.distance_fast(s1, s2, use_pruning=True)
print(d)

1.4142135623730951


In [36]:
s1 = np.array([[0, 0],
               [0, 1],
               [2, 1],
               [0, .5],
               [0, 0]], dtype=np.double)
s2 = np.array([[0, 0],
               [2, 1],
               [8, 1],
               [10, 0]], dtype=np.double)
d = dtw_ndim.distance(s1, s2, use_pruning=True)

print(d)

12.854960132182441


In [39]:
from dtaidistance import dtw_ndim

def nDTW(s1,s2):
  """
  Normalized Dynamic Time Warping as in "G.Ilharco et al. 2019"
    s1 = sequence 1 
    s2 = sequence 2 
    d_th = success distance threshold
  
  """
  norm_ = s1.shape[0]
  DTW = dtw_ndim.distance(s1, s2, use_pruning= True)
  d_th = norm_
  
  return np.exp(-(DTW)/(norm_*d_th))


  

In [40]:
nDTW(s1,s2)

0.597979739950246